In [13]:
!pip install wget tensorflow-gpu tensorflow_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-10-13 12:28:44--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-10-13 12:28:44 (110 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [17]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip")

unzip_data("nlp_getting_started.zip")

/bin/bash: -c: line 0: syntax error near unexpected token `)'
/bin/bash: -c: line 0: `wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip")'


In [15]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data

In [8]:
# Importing TensorFlow and Keras libraries
import tensorflow as tf
from keras import Sequential
from keras.layers import Input, GlobalAveragePooling1D, Dense, LSTM, Conv1D, GlobalMaxPool1D
from keras.layers import TextVectorization, Embedding
from keras.optimizers import Adam
import tensorflow_hub as hub

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd


In [9]:
class PreprocessData:

    def __init__(self):
        self.train_df = pd.read_csv('train.csv')
        self.test_df = pd.read_csv('test.csv')

        self.train_df_shuffled = self.train_df.sample(frac=1, random_state=42)

        self.train_sentences = None
        self.train_labels = None
        self.val_sentences = None
        self.val_labels = None

        self.max_vocab_length = 10000
        self.max_output_sequence_length = 15

    def get_train_val_data(self):

        self.train_sentences, self.val_sentences, self.train_labels, self.val_labels = train_test_split(self.train_df_shuffled['text'].to_numpy(),
                                                                                                        self.train_df_shuffled['target'].to_numpy(),
                                                                                                        test_size=0.1,
                                                                                                        random_state=42)
        return self.train_sentences, self.train_labels, self.val_sentences, self.val_labels

    def get_test_dataframe(self):

        return self.test_df


In [10]:
    # Initialize dataset for this file
    preprocess_data = PreprocessData()

    train_sentences, train_labels, val_sentences, val_labels = preprocess_data.get_train_val_data()

    max_vocab_length = preprocess_data.max_vocab_length
    max_output_length = preprocess_data.max_output_sequence_length


In [6]:
def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall and f1-score of a binary classification model;
    :param y_true: true labels in the form of a 1D array;
    :param y_pred: predicted labels in the form of a 1D array;
    :return: a dictionary of accuracy, precision, recall, f1-score.
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall and f1-score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1-score": model_f1}

    return model_results

In [11]:
    '''Exercise - 3'''

    # Creating tensorflow hub model encoding layer
    sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                            input_shape=[],
                                            dtype=tf.string,
                                            trainable=True,
                                            name="USE")

    # Building Sequential of model
    ex_model_use = Sequential([
        sentence_encoder_layer,
        Dense(64, activation="relu"),
        Dense(1, activation="sigmoid")
    ], name="model_use")

    # Compile the model
    ex_model_use.compile(loss="binary_crossentropy",
                         optimizer=Adam(),
                         metrics=["accuracy"])

    # Training the model
    ex_model_use_history = ex_model_use.fit(train_sentences,
                                            train_labels,
                                            epochs=5,
                                            validation_data=(val_sentences, val_labels))

    print("|=== Model USE Results ===|")  
    # Getting prediction probabilities
    ex_model_use_pred_probs = ex_model_use.predict(val_sentences)
    print(ex_model_use_pred_probs[:10])

    # Convert probabilities to labels (numbers)
    ex_model_use_preds = tf.squeeze(tf.round(ex_model_use_pred_probs))
    print(ex_model_use_preds[:20])

    # Calculate model results (accuracy, precision, recall, f1-score)
    ex_model_use_results = calculate_results(val_labels, ex_model_use_preds)


Epoch 1/5
215/215 [==============================] - 38s 152ms/step - loss: 0.4565 - accuracy: 0.7971 - val_loss: 0.4324 - val_accuracy: 0.8123
Epoch 2/5
215/215 [==============================] - 33s 153ms/step - loss: 0.2441 - accuracy: 0.9059 - val_loss: 0.5464 - val_accuracy: 0.7520
Epoch 3/5
215/215 [==============================] - 32s 150ms/step - loss: 0.0834 - accuracy: 0.9733 - val_loss: 0.6944 - val_accuracy: 0.8031
Epoch 4/5
215/215 [==============================] - 32s 150ms/step - loss: 0.0483 - accuracy: 0.9848 - val_loss: 0.7043 - val_accuracy: 0.7585
Epoch 5/5
215/215 [==============================] - 32s 150ms/step - loss: 0.0293 - accuracy: 0.9879 - val_loss: 0.6871 - val_accuracy: 0.7861
|=== Model USE Results ===|
24/24 [==============================] - 1s 9ms/step
[[0.22302952]
 [0.20386946]
 [0.99914   ]
 [0.01032338]
 [0.9751413 ]
 [0.9982004 ]
 [0.9991322 ]
 [0.99924064]
 [0.99896526]
 [0.01331461]]
tf.Tensor([0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0.